<a href="https://colab.research.google.com/github/avyaktawrat/3D_Reconstruction/blob/master/Week1_T2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following links a CNN that has been implemented using Caffe, Your task to implement the same network using Pytorch:

(a)  https://kushalvyas.github.io/caffe_cnn.html

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import numpy as np

In [3]:
class SubclassModel(nn.Module): 
  def __init__(self,num_classes): 
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels= 3, out_channels=96, kernel_size=11, stride=4, padding=0)
    ## the result on each output channel is calculated from the convolution kernel 
    ## corresponding to that output channel and takes input from all channels in the input array

    self.conv2 = nn.Conv2d(in_channels=  96, out_channels= 256, kernel_size= 5, stride= 1, padding= 2)
    self.conv3 = nn.Conv2d(in_channels= 256, out_channels= 384, kernel_size= 3, stride= 1, padding= 1)
    self.conv4 = nn.Conv2d(in_channels= 384, out_channels= 384, kernel_size= 5, stride= 1, padding= 2)
    self.conv5 = nn.Conv2d(in_channels= 384, out_channels= 256, kernel_size= 5, stride= 1, padding= 2)

    self.activation = nn.ReLU()
    self.norm = nn.LocalResponseNorm(size=2)
    self.pool = nn.MaxPool2d(kernel_size = 3,stride = 2,padding = 0)

    self.fc6 = nn.Linear(256,4096)           
    self.fc7 = nn.Linear(4096,4096)
    self.fc8 = nn.Linear(4096,1000)
    
    self.prob = nn.Softmax()                  ############
    self.drop = nn.Dropout()


  def forward(self, input): 
    conv_1 = self.conv1(input)              ## 3 X 96 X 96 --> 96 X 22 X 22
    activated_1 = self.activation(conv_1)   
    pool_1 = self.pool(activated_1)         ## --> 96 X 10 X 10
    norm_1 = self.norm(pool_1)

    conv_2 = self.conv2(norm_1)             ## --> 256 X 10 X 10
    activated_2 = self.activation(conv_2)
    pool_2 = self.pool(activated_2)         ## --> 256 X 4 X 4
    norm_2 = self.norm(pool_2)

    conv_3 = self.conv3(norm_2)             ## --> 384 X 4 X 4
    activated_3 = self.activation(conv_3)

    conv_4 = self.conv4(activated_3)        ## --> 384 X 4 X 4
    activated_4 = self.activation(conv_4)

    conv_5 = self.conv5(activated_4)        ## --> 256 X 4 X 4
    activated_5 = self.activation(conv_5)
    pool_5 = self.pool(activated_5)         ## --> 256 X 1 X 1
    
    fc_6 = self.fc6(pool_5)
    drop_6 = self.drop(fc_6)
    activated_6 = self.activated(drop_6)

    fc_7 = self.fc7(activated_6)
    drop_7 = self.drop(fc_7)
    activated_7 = self.activated(drop_7)

    fc_8 = self.fc8(activated_7)

    output = self.prob(fc_8,dim=1)
    return output
AlexNET = SubclassModel(2) 
AlexNET

#output = (input - kernel_size) / stride + 1

SubclassModel(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv5): Conv2d(384, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (activation): ReLU()
  (norm): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc6): Linear(in_features=256, out_features=4096, bias=True)
  (fc7): Linear(in_features=4096, out_features=4096, bias=True)
  (fc8): Linear(in_features=4096, out_features=1000, bias=True)
  (prob): Softmax(dim=None)
  (drop): Dropout(p=0.5, inplace=False)
)

In [0]:
def outputSize(in_size, kernel_size, stride, padding):  ## in size is spatial dimension, not channel
  output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
  return(output)
def poolSize(Input, Pool, Stride, Padding):
  output = int((Input - Pool + 2*Padding)/Stride) + 1
  return(output)

In [5]:
print(outputSize(96, 11, 4, 0))
print(poolSize(22, 3, 2, 0))

print(outputSize(10, 5, 1, 2))
print(poolSize(10, 3, 2, 0))

print(outputSize(4, 3, 1, 1))

print(outputSize(4, 5, 1, 2))

print(outputSize(4, 5, 1, 2))
print(poolSize(4, 3, 2, 0))

22
10
10
4
4
4
4
1


Loading the dataset

In [6]:
import tensorflow_datasets as tfds
import numpy as np

dataset, pcam_info = tfds.load("patch_camelyon", with_info=True, split=['train','test'], shuffle_files=True)
print(pcam_info)  

# print(dataset)
# (img_train, label_train), (img_test, label_test) = tfds.as_numpy(dataset)
train, test = tfds.as_numpy(dataset)

tfds.core.DatasetInfo(
    name='patch_camelyon',
    version=2.0.0,
    description='The PatchCamelyon benchmark is a new and challenging image classification
dataset. It consists of 327.680 color images (96 x 96px) extracted from
histopathologic scans of lymph node sections. Each image is annoted with a
binary label indicating presence of metastatic tissue. PCam provides a new
benchmark for machine learning models: bigger than CIFAR10, smaller than
Imagenet, trainable on a single GPU.
',
    homepage='https://patchcamelyon.grand-challenge.org/',
    features=FeaturesDict({
        'id': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(96, 96, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    }),
    total_num_examples=327680,
    splits={
        'test': 32768,
        'train': 262144,
        'validation': 32768,
    },
    supervised_keys=('image', 'label'),
    citation="""@misc{b_s_veeling_j_linmans_j_winkens_t_cohen_2018

In [7]:
k = 0
for i in train:
  # `{'image': np.array(shape=(96,96,3)), 'labels': np.array(shape=())}`
  
 
  i['image'] = torch.from_numpy(i['image'])
  i['image'] = (i['image'].transpose(0,2)).unsqueeze(0)

  i['label'] = torch.from_numpy(np.array(i['label']))
  i['label'] = (i['label']).unsqueeze(0)

  if k == 0:
    X_train = i['image']
    y_train = i['label']
  else:
    X_train = torch.cat([X_train, i['image']], dim = 0)
    # print(i['image'])
    # print(i['image'].size())

  
    y_train = torch.cat([y_train, i['label']], dim = 0)
  if k == 1000:
    break

  k = k + 1

# print (y_train)
print(X_train.size())



###############################
k = 0
for i in test:
  # `{'image': np.array(shape=(96,96,3)), 'labels': np.array(shape=())}`
  
 
  i['image'] = torch.from_numpy(i['image'])
  i['image'] = (i['image'].transpose(0,2)).unsqueeze(0)

  i['label'] = torch.from_numpy(np.array(i['label']))
  i['label'] = (i['label']).unsqueeze(0)

  if k == 0:
    X_test = i['image']
    y_test = i['label']
  else:
    X_test = torch.cat([X_test, i['image']], dim = 0)
    # print(i['image'])
    # print(i['image'].size())

  
    y_test = torch.cat([y_test, i['label']], dim = 0)
  if k == 100:
    break

  k = k + 1

# print (y_test)
print(X_test.size())

torch.Size([1001, 3, 96, 96])
torch.Size([101, 3, 96, 96])


Training the model

In [0]:
def training_model(n_epochs, optimizer, model, loss_fn, X_train, y_train, X_test, y_test):
  for epoch in range(1,n_epochs+1):
    y_pred_train = model(X_train)
    loss_train = loss_fn(y_pred_train, y_train)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()  ### updates all parameters

    if epoch == 1 or epoch % 1000 ==0 :
      y_pred_test = model(X_test)
      loss_test = loss_fn(y_pred_test, y_test)
      
      correct = (y_pred_test == y_test).sum(dim = 0)
      total = y_test.size(0)
      print('Epoch {}, Training loss {}, Validation loss {}'.format( epoch, float(loss_train), float(loss_val)))
      print('Test Accuracy of the model on the test images: {} %'.format((correct / total) * 100))

In [0]:
n = 1000

loss_func = nn.MSELoss()

In [0]:
optimizer = optim.SGD(AlexNET.parameters(),lr = 1e-2)

# training_model(n, optimizer, AlexNET, loss_func, X_train, y_train, X_test, y_test)